In [1]:
## a simple ML model use a person's age and experience to predict his/her salary
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Missing').getOrCreate()

23/03/04 20:41:47 WARN Utils: Your hostname, Huis-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.71.223 instead (on interface en0)
23/03/04 20:41:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/04 20:41:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
training = spark.read.csv('MLtest.csv',header=True,inferSchema=True)

In [3]:
training.show()


+-----+---+----------+----------+------+
| Name|Age|Department|Experience|Salary|
+-----+---+----------+----------+------+
| Abby| 18|     Admin|         1| 20000|
|  Bob| 20|       IOT|         1| 30000|
|Cathy| 31|       IOT|         5| 40000|
| Dave| 50|     Admin|        20| 60000|
|Frank| 40|      Data|        10| 50000|
|Grace| 45|     Admin|        15| 30000|
+-----+---+----------+----------+------+



In [4]:
training.columns

['Name', 'Age', 'Department', 'Experience', 'Salary']

In [46]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Department: string (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [5]:
## change categorical features into numbers
from pyspark.ml.feature import StringIndexer

In [9]:
indexer=StringIndexer(inputCol="Department",outputCol="dept_indexed")
df_r=indexer.fit(training).transform(training)
df_r.show()

+-----+---+----------+----------+------+------------+
| Name|Age|Department|Experience|Salary|dept_indexed|
+-----+---+----------+----------+------+------------+
| Abby| 18|     Admin|         1| 20000|         0.0|
|  Bob| 20|       IOT|         1| 30000|         1.0|
|Cathy| 31|       IOT|         5| 40000|         1.0|
| Dave| 50|     Admin|        20| 60000|         0.0|
|Frank| 40|      Data|        10| 50000|         2.0|
|Grace| 45|     Admin|        15| 30000|         0.0|
+-----+---+----------+----------+------+------------+



In [52]:
## VectorAssember : group columns into independent feature
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=["Age", "Experience"],outputCol="independent feature")

In [55]:
output=featureassembler.transform(training)
output.show()

+-----+---+----------+----------+------+-------------------+
| Name|Age|Department|Experience|Salary|independent feature|
+-----+---+----------+----------+------+-------------------+
| Abby| 18|     Admin|         1| 20000|         [18.0,1.0]|
|  Bob| 20|       IOT|         1| 30000|         [20.0,1.0]|
|Cathy| 31|       IOT|         5| 40000|         [31.0,5.0]|
| Dave| 50|     Admin|        20| 60000|        [50.0,20.0]|
|Frank| 40|      Data|        10| 50000|        [40.0,10.0]|
|Grace| 45|     Admin|        15| 30000|        [45.0,15.0]|
+-----+---+----------+----------+------+-------------------+



In [56]:
## input feature: 'independent feature', output feature: 'salary'
finalized_data=output.select("independent feature", "Salary")

In [57]:
finalized_data.show()

+-------------------+------+
|independent feature|Salary|
+-------------------+------+
|         [18.0,1.0]| 20000|
|         [20.0,1.0]| 30000|
|         [31.0,5.0]| 40000|
|        [50.0,20.0]| 60000|
|        [40.0,10.0]| 50000|
|        [45.0,15.0]| 30000|
+-------------------+------+



In [60]:
from pyspark.ml.regression import LinearRegression
## training test slipt
train_data, test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='independent feature', labelCol='Salary')
regressor=regressor.fit(train_data)

23/03/04 20:10:29 WARN Instrumentation: [be25e7fc] regParam is zero, which might cause numerical instability and overfitting.
23/03/04 20:10:29 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/03/04 20:10:29 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
23/03/04 20:10:29 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [61]:
regressor.coefficients

DenseVector([908.9725, -99.8871])

In [62]:
regressor.intercept

7488.491270737647

In [64]:
## prediction
pred_results=regressor.evaluate(test_data)

In [66]:
pred_results.predictions.show()

+-------------------+------+-----------------+
|independent feature|Salary|       prediction|
+-------------------+------+-----------------+
|         [31.0,5.0]| 40000|35167.20229305997|
+-------------------+------+-----------------+



In [67]:
pred_results.meanSquaredError, pred_results.degreesOfFreedom

(23355933.676204808, -2)